In [ ]:
!pip install dash
!pip install dash-bootstrap-components

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

In [ ]:
# Load the data
df = pd.read_csv("/content/OnlineRetail.csv", encoding='ISO-8859-1')

In [ ]:
# Remove rows where CustomerID is missing (needed for customer-level analysis)
df.dropna(subset=["CustomerID"], inplace=True)

# Create a new column for total price per transaction (Quantity * UnitPrice)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Convert InvoiceDate column to datetime format for time-based analysis
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True)
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

In [ ]:
# Calculate KPIs
kpi_total_revenue = df['TotalPrice'].sum()
kpi_total_invoices = df['InvoiceNo'].nunique()
kpi_total_customers = df['CustomerID'].nunique()
kpi_avg_recency = (df['InvoiceDate'].max() - df.groupby('CustomerID')['InvoiceDate'].max()).dt.days.mean()

In [ ]:
# Top 10 Products
top_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10).reset_index()

In [ ]:
# Revenue over time
revenue_over_time = df.groupby('InvoiceMonth')['TotalPrice'].sum().reset_index()
revenue_over_time['InvoiceMonth'] = revenue_over_time['InvoiceMonth'].astype(str)

In [ ]:
# Country sales distribution
country_sales = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).reset_index()

In [ ]:
# App setup
app = Dash(__name__)
app.title = "Online Retail Dashboard"

In [ ]:
# App layout
app.layout = dbc.Container([
    html.H1("SmartRetail360 - Sales & Customer Analytics Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col(dbc.Card([dbc.CardBody([html.H4("Total Revenue"), html.H2(f"\u00a3{kpi_total_revenue:,.0f}")])]), width=3),
        dbc.Col(dbc.Card([dbc.CardBody([html.H4("Total Invoices"), html.H2(f"{kpi_total_invoices}")])]), width=3),
        dbc.Col(dbc.Card([dbc.CardBody([html.H4("Unique Customers"), html.H2(f"{kpi_total_customers}")])]), width=3),
        dbc.Col(dbc.Card([dbc.CardBody([html.H4("Avg. Recency (Days)"), html.H2(f"{kpi_avg_recency:.0f}")])]), width=3),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([
            html.H5("Revenue Over Time"),
            dcc.Graph(figure=px.line(revenue_over_time, x='InvoiceMonth', y='TotalPrice', title='Monthly Revenue'))
        ], width=6),

        dbc.Col([
            html.H5("Top 10 Selling Products"),
            dcc.Graph(figure=px.bar(top_products, x='Quantity', y='Description', orientation='h', title='Top Products'))
        ], width=6),
    ]),

    dbc.Row([
        dbc.Col([
            html.H5("Sales by Country"),
            dcc.Graph(figure=px.pie(country_sales, values='TotalPrice', names='Country', title='Country Sales Share'))
        ], width=6),
    ])

], fluid=True)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)